### 드라이브 마운트

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 패키지 설치

In [26]:
!pip install openai langchain

In [27]:
from pprint import pprint
from typing import Dict, List

from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key

In [28]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


### Prompt chain 준비
* 서비스할 내용의 프롬프트 체인을 준비합니다.
* 각 프롬프트 체인을 미리 준비해 놓고, 템플릿으로 사용합니다.

In [29]:
P_PATH = "/content/drive/MyDrive/dataset/Novel_generation/multi_prompt"
IDEA_P = os.path.join(P_PATH, "extract_idea.txt")
OUTLINE_P = os.path.join(P_PATH, "write_outline.txt")
PLOT_P = os.path.join(P_PATH, "write_plot.txt")
CHAPTER_P = os.path.join(P_PATH, "write_chapter.txt")

### Prompt chain 구현
* `SequentialChain`을 이용해서 여러개의 chain을 연속적으로 구현할 수 있습니다.

In [54]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]]
    text: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path),
        ),
        output_key=output_key,
        verbose=True,
    )


def generate_novel(req: UserRequest) -> Dict[str, str]:
    writer_llm = ChatOpenAI(temperature=0.3, max_tokens=500, model="gpt-3.5-turbo")

    # 아이디어 뽑기 체인 생성
    novel_idea_chain = create_chain(writer_llm,IDEA_P, "novel_idea")

    # 아웃라인 작성 체인 생성
    novel_outline_chain = create_chain(
        writer_llm, OUTLINE_P, "novel_outline"
    )

    # 플롯 작성 체인 생성
    novel_plot_chain = create_chain(writer_llm, PLOT_P, "novel_plot")

    # 챕터 작성 체인 생성
    novel_chapter_chain = create_chain(writer_llm, CHAPTER_P, "output")

    preprocess_chain = SequentialChain(
        chains=[
            novel_idea_chain,
            novel_outline_chain,
            novel_plot_chain,
        ],
        input_variables=["genre", "characters", "text"],
        output_variables=["novel_idea", "novel_outline", "novel_plot"],
        verbose=True,
    )

    context = req.dict()
    context = preprocess_chain(user_data)

    context["novel_chapter"] = []
    for chapter_number in range(1, 4):
        context["chapter_number"] = chapter_number
        context = novel_chapter_chain(context)
        context["novel_chapter"].append(context["output"])

    contents = "\n\n".join(context["novel_chapter"])
    return {"results": contents}

### User prompt 작성
* User가 직접 작성하는 프롬프트를 작성합니다.

In [55]:
user_data = {
    "genre": "판타지",
    "characters": [
        {
            "name": "김철수",
            "role": "주인공"
        },
        {
            "name": "이영희",
            "role": "조연"
        }
    ],
    "text": "날씨가 추워지고 있습니다."
}


* User Prompt를 입력합니다.

In [56]:
request_instance = UserRequest(**user_data)

### Text Generation

In [57]:
generate_novel(request_instance)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[등장 인물] 과 [참고 텍스트] 를 소재로 새롭고 흥미진진한 판타지 소설 아이디어를 한 문단으로 작성해줘

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[아이디어]
한 평범한 겨울날, 주인공 김철수는 추운 날씨에 어울리는 따뜻한 커피를 마시며 도시를 거닐고 있었다. 그런데 갑자기 이영희라는 조연이 등장하며, 그녀는 마법사였다. 이영희는 김철수에게 날씨를 조절할 수 있는 마법을 가르쳐주었고, 그들은 함께 도시를 눈으로 뒤덮어버리는 환상적인 눈보라를 만들어냈다. 이 눈보라는 사람들에게 희망과 기쁨을 전달하며, 도시 전체가 환호성을 지르는 풍경을 만들었다. 이렇게 김철수와 이영희는 마법을 이용하여 날씨를 변화시키는 모험을 떠나게 되었고, 그들은 도시를 더욱 아름답게 만들어가는 여정에 도전하게 된다.

[context]
아웃라인 단계에서는 주요 이벤트와 결말을 고려하세요. 
여기서 중요한 것은, 이 단계에서 구체적인 디테일에 매몰되기보다는 스토리의 큰 그림에 집중하는 것입니다.

[등장 인물] 과 [아이디어] 를 소재로 새롭고 흥미진진한 판타지 소설의 아웃라인을 작성해줘


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 

{'results': '마법의 만남\n\n김철수는 추운 겨울날 도시를 거닐고 있었다. 얼어붙을 듯한 바람이 그의 얼굴을 때리며, 그는 손을 주머니에 넣어 따뜻함을 찾으려 했다. 그런데 갑자기 한 소녀가 그의 앞에 나타났다.\n\n"안녕하세요! 제 이름은 이영희예요. 마법사이고, 날씨를 조절할 수 있는 마법을 가지고 있어요." 이영희는 밝은 미소로 김철수에게 소개했다.\n\n김철수는 놀라움에 입을 다물지 못했다. 마법사? 날씨를 조절할 수 있는 마법? 그런 일은 동화나 영화에서만 본 것 같은데...\n\n이영희는 김철수의 궁금한 눈빛을 보고 웃음을 터뜨렸다. "괜찮아요, 저는 당신에게 그런 마법을 가르쳐줄 수 있어요. 함께 도시를 눈으로 뒤덮어버리는 환상적인 눈보라를 만들어볼까요?"\n\n김철수는 마법에 대한 흥미로 가슴이 뛰었다. 그는 이영희와 함께 모험을 떠나기로 결심했다. 이들은 도시 한복판에서 마법을 사용하여 눈보라를 일으키기 시작했다.\n\n눈은 하늘에서 쏟아져 내리며 도시를\n\n환상적인 눈보라\n\n김철수와 이영희는 함께 도시를 눈으로 뒤덮어버리는 환상적인 눈보라를 만들어냈습니다. 이들의 마법은 도시 전체에 희망과 기쁨을 전달하며, 사람들은 눈보라 속에서 환호성을 지르며 즐거움을 느꼈습니다.\n\n김철수와 이영희는 눈보라를 만들기 위해 도시의 중심부로 이동했습니다. 그들은 마법의 힘으로 하늘을 어둡게 만들고, 눈이 내리도록 조절했습니다. 처음에는 작은 눈송이가 하늘에서 춤을 추며 내리기 시작했습니다. 그리고 점점 눈송이의 크기가 커지며, 도시는 흰 눈으로 뒤덮여갔습니다.\n\n사람들은 눈보라 속에서 놀라움과 아름다움을 느끼며, 김철수와 이영희의 마법에 감탄했습니다. 눈송이들은 다양한 모양과 크기로 변화하며, 거리는 환상적인 풍경으로 변모했습니다. 사람들은 눈송이들을 잡으며 즐거움을 느끼고, 눈으로 뒤덮인 도시에서의 산책이 새로운 경험이 되었습니다.\n\n김철수와 이영희는 눈보라의 효과를 보고 더 많은 사람들에게\n\n도시 아름답게 만들기\n\n김철수